In [ ]:
import numpy as np

from experiments import *
import autoencoders as ae
import wandb

In [ ]:
wandb.init(project="test")

In [ ]:
suite = ae.TrajectoryAutoencoderSuite(Pendulum, additional_loss=lambda _, __ , model: 10 * model.contrastive_loss(), batch_size=512, epochs=1000, is_contrastive=True)
ree = suite.contrastive_learning()
model = ree.model

In [ ]:
suite.test(ree, Pendulum.contrastive_data()[:1000], 2)

In [ ]:
a = Pendulum.contrastive_data()[:1000]

In [ ]:
ee = ree(ae.torch.tensor(a).to(suite.device).float().view(-1, 3))
ee_np = ee.detach().cpu().numpy()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(a[:, 1], a[:, 2], c='blue', s=1)
plt.scatter(ee_np[:, 0], ee_np[:, 1], c='orange', s=1)
plt.show()